# Overview: 

This notebook is created to do tasks given in issue no #8: [Importance score for dataset training samples](https://github.com/mozilla/PRESC/issues/8)

# Task:
- Implement a way to assess the importance of an inidividual training datapoint to the performance of the model. 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
sns.set_style("whitegrid")

In [2]:
dataset = pd.read_csv("../../datasets/defaults.csv")
print(dataset.head())
print(f"Amount of samples: {dataset.shape[0]}")

   id  limit_bal  sex  education  marriage  age  pay_0  pay_2  pay_3  pay_4  \
0   1      20000    2          2         1   24      2      2     -1     -1   
1   2     120000    2          2         2   26     -1      2      0      0   
2   3      90000    2          2         2   34      0      0      0      0   
3   4      50000    2          2         1   37      0      0      0      0   
4   5      50000    1          2         1   57     -1      0     -1      0   

   ...  bill_amt4  bill_amt5  bill_amt6  pay_amt1  pay_amt2  pay_amt3  \
0  ...          0          0          0         0       689         0   
1  ...       3272       3455       3261         0      1000      1000   
2  ...      14331      14948      15549      1518      1500      1000   
3  ...      28314      28959      29547      2000      2019      1200   
4  ...      20940      19146      19131      2000     36681     10000   

   pay_amt4  pay_amt5  pay_amt6  defaulted  
0         0         0         0          

# Data normalization

SVMs are guaranteed to converge at some point. However, when data is unscaled this can take some time. In this step, some of the features are rescaled to be within the range of [-1, 1]. In this step the id column is also dropped from the dataset since it doesn't provide any relevant information for this task.

In [3]:
# Normalize columns 12 to 23. BILL_AMT1 to BILL_AMT6 and PAY_AMT1 to PAY_AMT6
norm_columns = ["limit_bal", "pay_amt1", "pay_amt2", "pay_amt3", "pay_amt4", "pay_amt5", "pay_amt6",
                "bill_amt1", "bill_amt2", "bill_amt3", "bill_amt4", "bill_amt5", "bill_amt6", "age"]
for c in norm_columns:
    max_val = np.max(dataset[c])
    min_val = np.min(dataset[c])
    
    dataset[c] = (dataset[c] - min_val) * 2 / (max_val - min_val) - 1

dataset.drop(columns=["id"], inplace=True)
print(dataset.head())

   limit_bal  sex  education  marriage       age  pay_0  pay_2  pay_3  pay_4  \
0  -0.979798    2          2         1 -0.896552      2      2     -1     -1   
1  -0.777778    2          2         2 -0.827586     -1      2      0      0   
2  -0.838384    2          2         2 -0.551724      0      0      0      0   
3  -0.919192    2          2         1 -0.448276      0      0      0      0   
4  -0.919192    1          2         1  0.241379     -1      0     -1      0   

   pay_5  ...  bill_amt4  bill_amt5  bill_amt6  pay_amt1  pay_amt2  pay_amt3  \
0     -2  ...  -0.679724  -0.838704  -0.478043 -1.000000 -0.999182 -1.000000   
1      0  ...  -0.673560  -0.831852  -0.473031 -1.000000 -0.998813 -0.997768   
2      0  ...  -0.652725  -0.809060  -0.454144 -0.996525 -0.998219 -0.997768   
3      0  ...  -0.626382  -0.781274  -0.432630 -0.995421 -0.997603 -0.997322   
4      0  ...  -0.640274  -0.800735  -0.448639 -0.995421 -0.956443 -0.977680   

   pay_amt4  pay_amt5  pay_amt6  defau

# Splitting the dataset for training and validation


In [4]:
def trainTestSplit(dataset, valid_per=0.1):
    n_samples = dataset.shape[0] # Total number of samples
    n_val = int(valid_per * n_samples)
    
    indices = np.arange(0, n_samples) # Generate a big array with all indices
    np.random.shuffle(indices) # Shuffle the array, numpy shuffles inplace
    
    # Perform the splits
    x_train = dataset.iloc[indices[n_val:], :-1].values # Last column is the feature we want to predict
    y_train = dataset.iloc[indices[n_val:], -1].values
    x_test = dataset.iloc[indices[:n_val], :-1].values
    y_test = dataset.iloc[indices[:n_val], -1].values
    
    return x_train, y_train, x_test, y_test

# Machine learning model

**Assuming same class weightings**

Now, let's train an SVM classifier to predict weather the credit default will be paid next month. In this step we assume the same weight for both classes, assuming that the dataset is balanced.

We can see that the accuracy seems reasonably good, however just looking at this metric masks an important issue that is happening. If take a look at recall values for class 1 we can see that it is VERY low. This means that the classifier is taking an "easy" route and just tends to classify most of the samples as class 0 (the dominant class), while underperforming on the other class.

This shows that for unbalanced datasets just looking at accuracy isn't enough. We can think of an extreme example were 99% of the samples are of class 0 and just 1% of class 1. In this scenario if the classifier always predicted class 0 for any sample it would achieve an accuracy of 99%.

In [5]:
print("Getting new dataset split...")
# Get a dataset split for training and validation
x_train, y_train, x_test, y_test = trainTestSplit(dataset, 0.2)

print(f"Training on {x_train.shape[0]} samples...")
print("\n#### Linear SVM Results ####")
# Create Linear SVM model
lsvm = LinearSVC(max_iter=32000) # If we don't specify anything it assumed all classes have same weight
lsvm.fit(x_train, y_train)
y_pred = lsvm.predict(x_test)
linear_acc = lsvm.score(x_test, y_test)
print(f"Linear SVM Acc: {linear_acc*100} % - Validated on {y_test.shape[0]} samples")
print(classification_report(y_test, y_pred))

print("\n#### Polynomial SVM with Degree 3 Results ####")
# Create Polynomial SVM
svm = SVC(gamma='scale', kernel='poly', degree=3)
svm.fit(x_train, y_train)
poly_acc = svm.score(x_test, y_test)
y_pred = svm.predict(x_test)
print(f"Polynomial SVM Acc: {poly_acc*100} %")
print(classification_report(y_test, y_pred))

Getting new dataset split...
Training on 24000 samples...

#### Linear SVM Results ####
Linear SVM Acc: 80.58333333333333 % - Validated on 6000 samples
              precision    recall  f1-score   support

           0       0.81      0.99      0.89      4671
           1       0.77      0.18      0.29      1329

   micro avg       0.81      0.81      0.81      6000
   macro avg       0.79      0.58      0.59      6000
weighted avg       0.80      0.81      0.75      6000


#### Polynomial SVM with Degree 3 Results ####
Polynomial SVM Acc: 82.35 %
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4671
           1       0.69      0.36      0.48      1329

   micro avg       0.82      0.82      0.82      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.82      0.80      6000



## Assigning class weightings relative to sample number

By initializing the SVM classifiers with the parameter class_weights="balanced" each class is assigned a weight based on its number of samples. The actual calculation is done by `(n_samples / (n_classes * np.bincount(y))`. 

Let's repeat the test with the same data.

We can see that the accuracy took a hit, however the recall for class 1 improved greatly. Nonetheless, the precision for predicting class 1 got much worse. I'm not sure why this is the case.

In [6]:
print(f"Training on {x_train.shape[0]} samples...")
print("\n#### Linear SVM Results ####")
# Create Linear SVM model
lsvm = LinearSVC(max_iter=32000, class_weight="balanced") # Compute weight based on sample count per class
lsvm.fit(x_train, y_train)
y_pred = lsvm.predict(x_test)
linear_acc = lsvm.score(x_test, y_test)
print(f"Linear SVM Acc: {linear_acc*100} % - Validated on {y_test.shape[0]} samples")
print(classification_report(y_test, y_pred))

print("\n#### Polynomial SVM with Degree 3 Results ####")
# Create Polynomial SVM
svm = SVC(gamma='scale', kernel='poly', degree=3, 
          class_weight="balanced") # Compute weight based on sample count per class
svm.fit(x_train, y_train)
poly_acc = svm.score(x_test, y_test)
y_pred = svm.predict(x_test)
print(f"Polynomial SVM Acc: {poly_acc*100} %")
print(classification_report(y_test, y_pred))

Training on 24000 samples...

#### Linear SVM Results ####
Linear SVM Acc: 69.06666666666666 % - Validated on 6000 samples
              precision    recall  f1-score   support

           0       0.88      0.70      0.78      4671
           1       0.38      0.65      0.48      1329

   micro avg       0.69      0.69      0.69      6000
   macro avg       0.63      0.68      0.63      6000
weighted avg       0.77      0.69      0.71      6000


#### Polynomial SVM with Degree 3 Results ####
Polynomial SVM Acc: 77.05 %
              precision    recall  f1-score   support

           0       0.88      0.82      0.85      4671
           1       0.49      0.60      0.54      1329

   micro avg       0.77      0.77      0.77      6000
   macro avg       0.68      0.71      0.69      6000
weighted avg       0.79      0.77      0.78      6000



## Balancing the training dataset and using same class weightings

A third scenario we can try here is actually balancing the training dataset and using the same class weightings. This will be done by randomly dropping samples of the dominant class until we have the same number of samples per class.

We can observe that there's not much difference from using the class weightings from the previous run. In fact the results are so similar that both ways might be equivalent and the small changes might be just due differences in the dropped samples.

In [7]:
def balanceTrainSet(x_train, y_train):
    samples_per_class = np.bincount(y_train) # Count samples per class
    dom_class = np.argmax(samples_per_class) # Max class index
    min_class = np.argmin(samples_per_class) # Min class index
    n_min = samples_per_class[min_class] # Number of samples in min class
    
    # Get indices for the dominant and the minor class
    dom_indices = np.where(y_train == dom_class)[0]
    min_indices = np.where(y_train == min_class)[0]
    np.random.shuffle(dom_indices) # Shuffle dom_indices
    # Contatenate both indices, using only the same number of indices from dom_indices as in min_indices
    indices = np.concatenate([min_indices, dom_indices[:n_min]], axis=0)
    np.random.shuffle(indices)
    
    # Build the new training set
    new_x_train = x_train[indices]
    new_y_train = y_train[indices]
    
    return new_x_train, new_y_train
    
bal_x_train, bal_y_train = balanceTrainSet(x_train, y_train)

print(f"Training on {bal_x_train.shape[0]} samples...")
print("\n#### Linear SVM Results ####")
# Create Linear SVM model
lsvm = LinearSVC(max_iter=32000) # If we don't specify anything it assumed all classes have same weight
lsvm.fit(bal_x_train, bal_y_train)
y_pred = lsvm.predict(x_test)
linear_acc = lsvm.score(x_test, y_test)
print(f"Linear SVM Acc: {linear_acc*100} % - Validated on {y_test.shape[0]} samples")
print(classification_report(y_test, y_pred))

print("\n#### Polynomial SVM with Degree 3 Results ####")
# Create Polynomial SVM
svm = SVC(gamma='scale', kernel='poly', degree=3)
svm.fit(bal_x_train, bal_y_train)
poly_acc = svm.score(x_test, y_test)
y_pred = svm.predict(x_test)
print(f"Polynomial SVM Acc: {poly_acc*100} %")
print(classification_report(y_test, y_pred))

Training on 10614 samples...

#### Linear SVM Results ####
Linear SVM Acc: 69.43333333333334 % - Validated on 6000 samples
              precision    recall  f1-score   support

           0       0.88      0.71      0.78      4671
           1       0.39      0.65      0.48      1329

   micro avg       0.69      0.69      0.69      6000
   macro avg       0.63      0.68      0.63      6000
weighted avg       0.77      0.69      0.72      6000


#### Polynomial SVM with Degree 3 Results ####
Polynomial SVM Acc: 77.45 %
              precision    recall  f1-score   support

           0       0.88      0.83      0.85      4671
           1       0.49      0.59      0.54      1329

   micro avg       0.77      0.77      0.77      6000
   macro avg       0.68      0.71      0.69      6000
weighted avg       0.79      0.77      0.78      6000



## Performing 10 Rounds of Cross Validation

Now I will perform 10 rounds of cross validation using the SVM method with class weightings relative to the number of samples per class. After 10 rounds the average accuracy and the mean average micro f1-score is presented for both SVM models. Micro F1-score is used because the classes are imbalanced.

Observing the results I can conclude that the SVM model with polynomial kernel of degree 3 performs better than the Linear SVM model on this data.

In [8]:
N_ROUNDS = 10
l_reports = [] # Linear SVM reports
p_reports = [] # Polynomial SVM reports
l_accs = [] # Linear SVM accuracy history
p_accs = [] # Polynomial SVM accuracy history
for i in range(N_ROUNDS):
    print(f"### Round {i+1} ###")
    print("Getting new dataset split...")
    x_train, y_train, x_test, y_test = trainTestSplit(dataset, 0.2)    
    print(f"Training on {x_train.shape[0]} samples...")
    
    # Create a new Linear SVM model
    lsvm = LinearSVC(max_iter=32000, class_weight="balanced") # Compute weight based on sample count per class
    lsvm.fit(x_train, y_train)
    y_pred = lsvm.predict(x_test)
    linear_acc = lsvm.score(x_test, y_test)
    print(f"Linear SVM Acc: {linear_acc*100} % - Validated on {y_test.shape[0]} samples")
    report = classification_report(y_test, y_pred, output_dict=True)
    l_reports.append(report)
    l_accs.append(linear_acc)

    # Create Polynomial SVM
    svm = SVC(gamma='scale', kernel='poly', degree=3, 
              class_weight="balanced") # Compute weight based on sample count per class
    svm.fit(x_train, y_train)
    poly_acc = svm.score(x_test, y_test)
    y_pred = svm.predict(x_test)
    print(f"Polynomial SVM Acc: {poly_acc*100} %")
    report = classification_report(y_test, y_pred, output_dict=True)
    p_reports.append(report)
    p_accs.append(poly_acc)
    
print("### Finished ###")
print("Linear SVM Results:")
print(l_reports[0])
mean_acc = np.mean(l_accs)
mean_f1 = np.mean([r["weighted avg"]["f1-score"] for r in l_reports])
print(f"\tMean Acc: {mean_acc*100}% -- Mean Weighted Avg F1-Score: {mean_f1*100}%")

print("Polynomial SVM with Degree 3 Results:")
mean_acc = np.mean(p_accs)
mean_f1 = np.mean([r["weighted avg"]["f1-score"] for r in p_reports])
print(f"\tMean Acc: {mean_acc*100}% -- Mean Weighted Avg F1-Score: {mean_f1*100}%")

### Round 1 ###
Getting new dataset split...
Training on 24000 samples...
Linear SVM Acc: 71.55 % - Validated on 6000 samples
Polynomial SVM Acc: 78.16666666666666 %
### Round 2 ###
Getting new dataset split...
Training on 24000 samples...
Linear SVM Acc: 69.95 % - Validated on 6000 samples
Polynomial SVM Acc: 78.01666666666667 %
### Round 3 ###
Getting new dataset split...
Training on 24000 samples...
Linear SVM Acc: 69.68333333333334 % - Validated on 6000 samples
Polynomial SVM Acc: 77.13333333333333 %
### Round 4 ###
Getting new dataset split...
Training on 24000 samples...
Linear SVM Acc: 69.8 % - Validated on 6000 samples
Polynomial SVM Acc: 76.73333333333333 %
### Round 5 ###
Getting new dataset split...
Training on 24000 samples...
Linear SVM Acc: 69.46666666666667 % - Validated on 6000 samples
Polynomial SVM Acc: 77.71666666666667 %
### Round 6 ###
Getting new dataset split...
Training on 24000 samples...
Linear SVM Acc: 68.75 % - Validated on 6000 samples
Polynomial SVM Acc: 7

# Feature Selection

Here, we remove the feature with the least impact on generalization accuracy one-by-one, until there's no remaining features to be removed. The goal is to find whether some features are irrelevant and how much they impact the accuracy of the model. This task will be performed on the same data throughout and using the polynomial of degree 3 model.

This reveals a very interesting pattern. Using even one feature we can achieve a very similar performance (even slightly better).

In [9]:
FEATURE_NAMES = ["limit_bal", "sex", "education", "marriage", "age", 
                 "pay_0", "pay_2", "pay_3", "pay_4", "pay_5", "pay_6", 
                 "bill_amt1", "bill_amt2", "bill_amt3", "bill_amt4", "bill_amt5", "bill_amt6", 
                 "pay_amt1", "pay_amt2", "pay_amt3", "pay_amt4", "pay_amt5", "pay_amt6"]

# Due to computing time constraints use a much smaller training dataset here
x_train, y_train, x_test, y_test = trainTestSplit(dataset, 0.85)

def trainModel(x_train, y_train, x_test, y_test):
    svm = SVC(gamma='scale', kernel='poly', degree=3, 
              class_weight="balanced")
    svm.fit(x_train, y_train)
    return svm.score(x_test, y_test)

print(f"Will train on {x_train.shape[0]} samples and validate on {x_test.shape[0]} samples.")
# Train a baseline model for this data, including all the features
baseline_acc = trainModel(x_train, y_train, x_test, y_test)
print(f"Baseline Acc: {baseline_acc*100}%")
print("====================================")

remaining_features = np.arange(0, x_train.shape[1])
for i in range(x_train.shape[1]-1):
    feat_names = [FEATURE_NAMES[r] for r in remaining_features]
    print(f"Remaining features: ", end=' ')
    [print(feat, end=', ') for feat in feat_names]
    print()
    
    best_acc = 0.0
    least_impact_feature = 0
    # Find feature with least impact on performance
    for c in range(remaining_features.shape[0]):
        # Test by removing each of the columns
        curr_features = np.delete(remaining_features, c)
        part_x_train = x_train[:, curr_features]
        part_x_test = x_test[:, curr_features]
        
        acc = trainModel(part_x_train, y_train, part_x_test, y_test)
        if acc > best_acc:
            best_acc = acc
            least_impact_feature = c

    print(f"Removing feature {FEATURE_NAMES[remaining_features[least_impact_feature]]} -- Had the least impact on performance (Acc: {best_acc*100} %)")
    remaining_features = np.delete(remaining_features, least_impact_feature)
    print("====================================")
    
print(f"Last feature is {FEATURE_NAMES[remaining_features[0]]}")
part_x_train = x_train[:, remaining_features]
part_x_test = x_test[:, remaining_features]
acc = trainModel(part_x_train, y_train, part_x_test, y_test)
print(f"Accuracy: {acc*100} %")

Will train on 4500 samples and validate on 25500 samples.
Baseline Acc: 77.08235294117647%
Remaining features:  limit_bal, sex, education, marriage, age, pay_0, pay_2, pay_3, pay_4, pay_5, pay_6, bill_amt1, bill_amt2, bill_amt3, bill_amt4, bill_amt5, bill_amt6, pay_amt1, pay_amt2, pay_amt3, pay_amt4, pay_amt5, pay_amt6, 
Removing feature pay_2 -- Had the least impact on performance (Acc: 77.64313725490196 %)
Remaining features:  limit_bal, sex, education, marriage, age, pay_0, pay_3, pay_4, pay_5, pay_6, bill_amt1, bill_amt2, bill_amt3, bill_amt4, bill_amt5, bill_amt6, pay_amt1, pay_amt2, pay_amt3, pay_amt4, pay_amt5, pay_amt6, 
Removing feature pay_3 -- Had the least impact on performance (Acc: 77.8235294117647 %)
Remaining features:  limit_bal, sex, education, marriage, age, pay_0, pay_4, pay_5, pay_6, bill_amt1, bill_amt2, bill_amt3, bill_amt4, bill_amt5, bill_amt6, pay_amt1, pay_amt2, pay_amt3, pay_amt4, pay_amt5, pay_amt6, 
Removing feature pay_6 -- Had the least impact on perform

# Clustering customers with K-Means
Here, K-Means is used to cluster the customers based on the features, without taking in account if the default was paid next month. Then, a single cluster is chosen to be considered as the class where the default was paid and the overall accuracy is calculated. The test is repeated for different cluster sizes.

We can see that using only 2 clusters yielded the best (although not good) result. This happens probably because as we use a larger number of clusters the data becomes more partitioned, i.e. there's less samples per cluster, so the accuracy takes a drop even though the probability is "better" in-cluster. An interesting extension of this is exhaustively searching through all combinations of clusters to find the one that yields the best overall accuracy.

In [10]:
# Get a dataset split for clustering and validation
x_train, y_train, x_test, y_test = trainTestSplit(dataset, 0.2)    

n_clusters = [2, 4, 8, 16, 32]
for n in n_clusters:
    print(f"K-Means with {n} clusters:")
    # Create K-Means model
    kmeans = KMeans(n_clusters=n)
    kmeans.fit(x_train) # Fit to training data
    # Get clusters from validation data
    clustered = kmeans.predict(x_test)
    # For each cluster, find the probability defaulting (that the client paid it next month)
    # by comparing the number of samples for each class
    highest_prob = 0.0
    highest_c = 0
    overall_acc = 0.0
    for c in range(n):
        # Retrieve samples that belong to the current cluster
        indices = np.where(clustered == c)[0]
        samples_in_cluster = [y_test[s] for s in indices]
        # How many of those samples are of customers with default paid next month?
        proportion = np.bincount(samples_in_cluster)
        prob = proportion[1] / np.sum(proportion)
        print(f"[Cluster {c}] - Probability of paid credit in this cluster: {prob*100} %")
        if prob > highest_prob:
            highest_c = c
            highest_prob = prob
            overall_acc = proportion[1] / y_test.shape[0]
    
    print(f"Choosing the single cluster {highest_c} as default_paid yields an overall Accuracy of {overall_acc*100} %")
    
    print()

K-Means with 2 clusters:
[Cluster 0] - Probability of paid credit in this cluster: 24.94342469197888 %
[Cluster 1] - Probability of paid credit in this cluster: 15.126050420168067 %
Choosing the single cluster 0 as default_paid yields an overall Accuracy of 16.53333333333333 %

K-Means with 4 clusters:
[Cluster 0] - Probability of paid credit in this cluster: 15.225845049356865 %
[Cluster 1] - Probability of paid credit in this cluster: 49.59016393442623 %
[Cluster 2] - Probability of paid credit in this cluster: 15.507411630558723 %
[Cluster 3] - Probability of paid credit in this cluster: 66.26506024096386 %
Choosing the single cluster 3 as default_paid yields an overall Accuracy of 4.583333333333333 %

K-Means with 8 clusters:
[Cluster 0] - Probability of paid credit in this cluster: 53.74280230326296 %
[Cluster 1] - Probability of paid credit in this cluster: 24.170616113744074 %
[Cluster 2] - Probability of paid credit in this cluster: 11.906193625977151 %
[Cluster 3] - Probabilit